In [2]:
import numpy as np
import torch
from torchvision import datasets
from torch.utils.data import random_split
from matplotlib import pyplot as plt


In [3]:
# Based on code from the pytorch tutorials

data_path = '../data/'

cifar10_train_val = datasets.CIFAR10(data_path, train=True, download=True)
cifar10_test = datasets.CIFAR10(data_path, train=False, download=True)

n_train = int(len(cifar10_train_val)*0.90)
n_val =  len(cifar10_train_val) - n_train
cifar10_train, cifar10_val = random_split(
        cifar10_train_val, 
        [n_train, n_val],
        generator=torch.Generator().manual_seed(123)
)

Files already downloaded and verified
Files already downloaded and verified
